# 代数計算(微分・積分・行列計算)を行う

代数計算ライブラリ`sympy`を用いて簡単な代数計算を行い、できることの一例を示します。

まずは必要なライブラリを`import`してきます。

In [1]:
from fumi_python_libraries import BasicAlgebraicCalculationClass
import numpy
import sympy

インスタンスを生成します。

In [2]:
alg_calc = BasicAlgebraicCalculationClass()

## 基本操作

### 出力形式の確認

まずはデフォルトの値で出力を見てみます。

`BasicAlgebraicCalculationClass`のクラスでは`print_formula()`という関数が実装されています。

`select_print_mode( bool )`で表示形式を判別する変数`latex_paste_mode`を設定できます。

デフォルトではこの変数は`False`で、`IPython.diplay.display()`で描画されます（Jupyter Notebookで数式を綺麗に出力してくれる関数です。）

一方で`True`では`print()`で出力します。この形式ではLatex(Overleafやcloud latex)、Keynoteの数式モードにペーストして使える形でダンプしてくれます。

In [3]:
alg_calc.print_formula()

<IPython.core.display.Math object>

`latex_paste_mode = True`にして見てみましょう。

In [4]:
alg_calc.select_print_mode(True)
alg_calc.print_formula()

f(x) = x \\


### よく使用するメソッドの使い方

#### 代数の宣言と代数計算を行う。

まずは宣言時ですが、引数として生成する変数リストを渡すことができます。

この際、生成した変数はクラス内の変数リストに格納されるので`get_variable_symbols()`を用いて取得することができます。

In [5]:
alg_calc = BasicAlgebraicCalculationClass(["\\alpha","\\beta","x"])

In [6]:
var = alg_calc.get_variable_symbols()

上記のようにして渡された変数は普通のプログラム同様に四則演算ができます。

In [7]:
fun = ( var[0] ) / ( var[2]**3 + var[1] )

このようにして計算した関数を`BasicAlgebraicCalculationClass`の右辺に渡しましょう。

メソッドは`set_right_hand_side(function)`です。

In [8]:
alg_calc.set_right_hand_side(fun)
alg_calc.print_formula()

<IPython.core.display.Math object>

同様にして、`set_left_hand_side(function)`左辺にも値を渡すことができます。

ただし、`set_left_hand_side(function)`の時渡す引数の方は文字列を前提にしています。

これは後述する行列計算の時におかしな描かれ方をすることを防ぐためです。

`sympy`で宣言したもの四則演算の結果を描画する際は代数を名前の順などにソートして出力します。

行列の代表させた大文字の$A$などの値をいれて$A^{-1}PA$などを描くと出力した時に$A^{-1}AP$などを表示してきます。

そのため少し面倒ですが、左辺は文字列で書くことにしています。

では同様にして左辺を代入してみましょう。

In [9]:
alg_calc.set_left_hand_side('f(\\alpha,\\beta,x)')
alg_calc.print_formula()

<IPython.core.display.Math object>

新たに変数を生成する場合は`create_variables()`を使用します。

In [10]:
var = alg_calc.create_variables(["a","b","x"])
alg_calc.set_left_hand_side('f(a,b,x)')
fun = ( var[0] ) / ( var[2]**3 + var[1] )
alg_calc.set_right_hand_side(fun)
alg_calc.print_formula()

<IPython.core.display.Math object>

またこのクラスが持つ左辺と右辺に変数に値を渡さずにダンプするメソッド`print_external_input(left, right)`もあります。

例えば以下のようできます。

In [11]:
alg_calc.print_external_input("x(t)", var[0] * sympy.sin ( var[2] ) + var[1] * sympy.sin ( var[2] ) + abs( var[0] + var[2]**var[1] ) )

<IPython.core.display.Math object>

念の為もう一度`print_formula()`を用いてクラス内の左辺と右辺の式を格納している変数が更新されていないことを確認します。

In [12]:
alg_calc.print_formula()

<IPython.core.display.Math object>

#### 数値を代入する

次に値を代入してみましょう。値を代入して出力する時は複数のメソッドを使用します。
まず、`set_variable_symbols(　list )`にはクラスが持つ変数のうち値を代入したり、微分積分をしたりする変数を指定します。

ここでは`a`, `b`を定数として、これらの変数に値を代入してみましょう。

In [13]:
alg_calc.set_variable_symbols(["a","b"])

上のメソッド自体は戻り値を持たないので、そのまま代入の操作を行なっていきます。

代入は`assign_values(variable list,valule list)`を使用します。

今回は$a=1$, $b=2$を代入してみます。

In [14]:
alg_calc.assign_values(["a","b"],[1,2])

上記の操作を行うと生成したインスタンス内の右辺ではない変数に一時的値が格納されます。

この一時的に格納された値は`get_variable_values()`を使用して取得できます。

以下のセルでは取得したのち、`print_external_input()`でダンプてみます。

In [15]:
alg_calc.print_external_input('\\mathrm{Asigned\\,function}', alg_calc.get_variable_values())

<IPython.core.display.Math object>

右辺を更新したい場合は、`set_right_hand_side()`に値を渡すか、`assign_values()`の第4引数を`True`にすると更新できます。（第3引数については丸め込みのフラグです。この後説明します。）

このように代入後に変数を更新しない理由はloop処理ないで毎回別の値を代入する時にいちいちloopの中でインスタンスを生成したり、関数を定義しなおしたりしなくても済むようにしているためです。

In [16]:
print("# 関数を再定義")
var = alg_calc.create_variables(["a","b","x"])
alg_calc.set_left_hand_side('f(a,b,x)')
fun = ( var[0] ) / ( var[2]**3 + var[1] )
alg_calc.set_right_hand_side(fun)
alg_calc.print_formula()
print("# 値を代入して更新")
alg_calc.set_variable_symbols(["a","b"])
alg_calc.assign_values(["a","b"],[1,2],None,True)
alg_calc.print_formula()

# 関数を再定義


<IPython.core.display.Math object>

# 値を代入して更新


<IPython.core.display.Math object>

#### 計算結果/出力結果を丸める

また`sympy`で出力すると、表示される桁がじどうで調整されないので割り切れない数字を入れると大変なことになったりします。

そのためいくつかの丸め込み用の引数やメソッドが用意されています。

例えば循環小数$\dfrac{1}{19}=0.\overline{052631578947368421}$を表示してみましょう。

In [17]:
function = var[0] / ( var[1] + var[2] )
values = [1, 19, 0]
labels = ["a", "b", "x"]
alg_calc.set_right_hand_side( function )
alg_calc.set_variable_symbols( labels )
alg_calc.assign_values( labels, values )
alg_calc.set_left_hand_side( "\\frac{1}{19}" )
alg_calc.set_right_hand_side( alg_calc.get_variable_values() )
alg_calc.print_formula()

<IPython.core.display.Math object>

こんな感じで長ったらしい数字が羅列されても困ってしまうことがあります。

これをなんとかする方法の一つとして、先ほど使用した代入するメソッド`assign_values()`の第3引数にて丸める桁指定できます。

書き方は`assign_values(variable list,valule list, round digit)`です。

このメソッドでは代入する前に丸めることになります。

例えば以下のような感じになります。

In [18]:
values = [1, 10.5000000001, 9.4999999999]
labels = ["a", "b", "x"]
alg_calc.set_right_hand_side( function )
alg_calc.set_variable_symbols( labels )
alg_calc.assign_values( labels, values , 6)
alg_calc.set_left_hand_side( "\\mathrm{Round}\\left(\\frac{1}{19},0\\right)" )
alg_calc.set_right_hand_side( alg_calc.get_variable_values() )
alg_calc.print_formula()

<IPython.core.display.Math object>

ただしこの場合は代入される前に丸められるので上記のように6桁目で丸めるようにしてもアウトプットは小数点2桁になってしまいます。

それもそのはずで、代入前に四捨五入しているので、$\frac{1}{10.5+9.5}=\frac{1}{20}$になっているので想定以上に丸め込まれていることがわかると思います。

このような場合には代入する前ではなく、後に丸めるたくなります。

そのためのメソッドが`round_right_hand_side( round_digits )`です。

In [19]:
function = var[0] / ( var[1] + var[2] )
values = [1, 19, 0]
labels = ["a", "b", "x"]
alg_calc.set_right_hand_side( function )
alg_calc.set_variable_symbols( labels )
alg_calc.assign_values( labels, values )
alg_calc.set_left_hand_side( "\\mathrm{Round}\\left(\\frac{1}{19},6\\right)" )
alg_calc.set_right_hand_side( alg_calc.get_variable_values() )
alg_calc.round_right_hand_side(6)
alg_calc.print_formula()

<IPython.core.display.Math object>

上記の`round_right_hand_side( round_digits )`では元々の右辺を更新するので計算を終えて、最後に表示する時のみ使用することが推奨されます。

#### ObjectをCopyする

さらにクラスをコピーするメソッドもあります。
このメソッドはモンテカルロなどにおいてloop処理ないで逐次代入する際にいちいち新しいクラスを宣言する必要がないようにするためです。

In [20]:
copied_alg_calc = alg_calc.copy_class()
copied_alg_calc.print_formula()

<IPython.core.display.Math object>

## 微分積分をおこなう

sympyが持つ微分積分をおこなうメソッドを用いた微分・積を行うメソッドが用意されています。

その方法を書いておきます。

### 不定積分をしてみる

不定積分は`indefinite_integral(variable, update flag)`を使います。

まずは積分する前に新たに積分するための関数をわたします。

一度初期化しても良いですが`set_formula（lhs, rhs)`で外部から一度に右辺と左辺を登録できます。

In [21]:
alg_calc.select_print_mode(False)
var = alg_calc.create_variables( ["a", "x", "b", "y", "c","\\alpha", "\\beta"] )
fun = var[0] * var[1] + var[2] * var[3]**var[5] + var[4]**var[6]
alg_calc.set_formula( 'f(x,y)', fun )
alg_calc.print_formula()

<IPython.core.display.Math object>

あとは`indefinite_integral(variable, update_flag)`で不定積分ができます。

第2引数は積分結果で右辺を更新するかを選択できます。

In [22]:
ind_intg = alg_calc.indefinite_integral( var[3], True )
alg_calc.print_formula()

<IPython.core.display.Math object>

### 定積分をしてみる

次に定積分です。こちらは`definite_integral( variable , min, max, update flag )`でできます。

In [23]:
int_range = [0, 2]
alg_calc.set_formula( 'f(x,y)', fun )
def_intg = alg_calc.definite_integral( var[1] , *int_range, True )
alg_calc.print_formula()

<IPython.core.display.Math object>

### 微分してみる

次に微分です。こちらは`differentiate( variable, update flag )`です。

In [24]:
alg_calc.set_formula( 'f(x,y)', fun )
dif_func = alg_calc.differentiate( var[4], True )
alg_calc.print_formula()

<IPython.core.display.Math object>

## 行列計算をしてみる。

最後に行列計算です。

行列の計算では生成するためのメソッドなどが少し異なります。

`create_martix_symbols( matrixsymbol, row, col, elementsymbol)`で生成します。

第四引数は行列の各要素を代表する記号を指定したい時に使用します。

デフォルトでは`None`になっており、この場合は第一引数で指定した文字の小文字が採用されます。

戻り値は行列そのものと各要素の代数を格納したリストです。

In [25]:
n = 10
A, a = alg_calc.create_martix_symbols( "A", n, n )
alg_calc.set_formula( 'A' , A )
alg_calc.print_formula()

<IPython.core.display.Math object>

第2の戻り値は行列の各要素に値を代入する時に使用されます。

以下では例えば、上記の$10 \times 10$の行列に対して後からヒルベルト行列を代入してみます。

まずはnumpyを使ってヒルベルト行列を数値で計算します。

In [26]:
def hilbert_matrix( n ):
  H = numpy.zeros( ( n, n ) )
  for i in range( n ):
    for j in range( n ):
      H[ i, j ] = 1 / ( i + j + 1 )
  return H
    
h_np = hilbert_matrix( n ).flatten()

上記の式では最後に行列を平滑化しています。

これは`assign_values()`で代入する際に渡すのものがリストのためです。

今回は代入した後に`sympy.nsimplify( function, rational=True ) )`を用いて分数にしてから出力しています。

これはみやすさ重視のためです。（ただし、結構適当に値の近い分数を取ってくることもあるので注意）

In [27]:
alg_calc.set_formula( 'A' , A )
alg_calc.assign_values( a, h_np, None, True )
h_sy = alg_calc.get_right_hand_side()
alg_calc.set_formula( 'H', sympy.nsimplify( h_sy, rational=True ) )
alg_calc.print_formula()

<IPython.core.display.Math object>

上記の処理において肝心の計算部分では数値のみを使っていたので、当然成功していてほしいわけです。

しかしsympyのいいところは代数のまま色々計算できるところです。

そのため最後に、完全に代数のみで対角化をしたのちに値を代入してみます。

今回は$2\times 2$の行列を用意します。

In [28]:
n = 2
L, l = alg_calc.create_martix_symbols( "Lambda", n, n )
alg_calc.set_formula( "\\Lambda" , L )
alg_calc.print_formula()

<IPython.core.display.Math object>

sympyの`diagonalize()`メソッドを使って対角化します。

戻り値は対角行列を$\Lambda=PDP^{-1}$とする時、$P$, $D$の順です。

In [29]:
p, d = alg_calc.right_hand_side.diagonalize()
alg_calc.print_external_input("P",p)
alg_calc.print_external_input("D",d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

上記のように対角化を計算できました。

もちろん得られた行列についても代数計算可能です。

まず、対角行列を計算してみましょう。

In [30]:
alg_calc.set_right_hand_side( p**-1 * L * p )
alg_calc.set_left_hand_side( 'P\\Lambda P^{-1}' )
alg_calc.print_formula()

<IPython.core.display.Math object>

結構形が複雑ですね。

自分で式を書いていたらかなり苦しそうです…

この対角行列に値を代入していきましょう。

In [31]:
l_np = hilbert_matrix( n ).flatten()
alg_calc.set_right_hand_side( p**-1 * L * p )
alg_calc.assign_values( l, l_np)
alg_calc.set_formula( 'PHP^{-1}', alg_calc.get_matrix( alg_calc.get_variable_values() ) )
alg_calc.round_right_hand_side(10)
alg_calc.print_formula()
php = alg_calc.get_right_hand_side()

<IPython.core.display.Math object>

描画を綺麗にするために10桁で丸めました。

さてこの値が正しいかを確認する方法ですが、$P$の行列（逆行列）をもう一度かけて値を代入した時にヒルベルト行列になっていれば良いはずです。

まずは行列$P$を上記の対角行列にかけます。

この際、下のセル内のpは代数なので出力結果には$\lambda_{ij}$が残ります。

In [32]:
pphpp = p * php * p**-1
alg_calc.print_external_input( 'P^{-1}\\left(PH P^{-1}\\right)P', pphpp )

<IPython.core.display.Math object>

よってもう一度、代入して分数の形で出力してみます。

In [33]:
alg_calc.set_right_hand_side( pphpp )
alg_calc.assign_values( l, l_np, None, True )
h_sy = alg_calc.get_right_hand_side()
alg_calc.set_formula( "H", sympy.nsimplify( h_sy, rational=True ) )
alg_calc.print_formula()

<IPython.core.display.Math object>

非常に良い結果を得ました。